##### 公車票證資料

In [11]:
import pandas as pd

In [12]:
# TO2
# data_file_TO2 = "/home/jmwu/桌面/MRT_Od/data/高雄市公車電子票證資料(TO2A)20240825/00743_sample.csv"
# bus_TO2 = pd.read_csv(data_file_TO2)

# # 調整資料
# bus_TO2.columns = bus_TO2.iloc[0]
# df_TO2 = bus_TO2[1: ]

# df_TO2.reset_index(drop = True, inplace = True)# 

In [14]:
# TO2A
data_file_TO2A = "/home/jmwu/桌面/MRT_Od/data/高雄市公車電子票證資料(TO2A)20240825/00743_sample.csv"
bus_TO2A = pd.read_csv(data_file_TO2A)


# 調整資料
bus_TO2A.columns = bus_TO2A.iloc[0]
df_TO2A = bus_TO2A[1: ]

df_TO2A.reset_index(drop = True, inplace = True)

##### 從原始資料中分出MeNGo的使用者
- KHH-399  ：TPASS月票
- KHH-MaaS-30D399：QRcode 月票

In [15]:
"""
Cell generated by Data Wrangler.
"""
# # 分出MeNGo的使用者
# # TO2
# def clean_data(df_TO2):
#     # Filter rows based on column: 'SubTicketType'
#     df_TO2 = df_TO2[df_TO2['SubTicketType'].str.contains("#KHH", regex = False, na = False)]
#     return df_TO2

# df_TO2_KHH = clean_data(df_TO2.copy())
# print(df_TO2_KHH.shape)

# TO2A
def clean_data(df_TO2A):
    # Filter rows based on column: 'SubTicketType'
    df_TO2A = df_TO2A[df_TO2A['SubTicketType'].str.contains("#KHH", regex = False, na = False)]
    return df_TO2A

df_TO2A_KHH = clean_data(df_TO2A.copy())

print(df_TO2A_KHH.shape)

(617, 33)


In [5]:
# print(df_TO2A_KHH.columns)

Index(['Authority', 'OperatorNo', 'ID', 'IDType', 'HolderType', 'TicketType',
       'SubTicketType', 'RouteUID', 'RouteName', 'SubRouteUID', 'SubRouteName',
       'Direction', 'FarePricingType', 'BoardingStopUID', 'BoardingStopName',
       'BoardingStopSequence', 'BoardingTime', 'DeboardingStopUID',
       'DeboardingStopName', 'DeboardingStopSequence', 'DeboardingTime',
       'Price', 'Discount', 'TransferCode', 'DiscountInfo', 'PaymentPrice',
       'IsAbnormal', 'ErrorCode', 'Result', 'TicketCount', 'SrcUpdateTime',
       'UpdateTime', 'InfoDate'],
      dtype='object', name=0)


In [6]:
# 兩個DataFrame的column有哪裡不同
columns_diff = set(df_TO2_KHH.columns).symmetric_difference(set(df_TO2A_KHH.columns))
print("Columns that differ between df_TO2_KHH and df_TO2A_KHH:")
print(columns_diff)

Columns that differ between df_TO2_KHH and df_TO2A_KHH:
{'SubRouteName', 'Direction', 'OperatorNo', 'SubRouteUID', 'RouteName', 'RouteUID'}


##### TO2跟TO2A的不同
- 'SubRouteUID'：搭乘附屬路線代碼
- 'RouteName'：搭乘路線名稱
- 'OperatorNo'：
- 'Direction'：搭乘公車路線方向
- 'RouteUID：搭乘路線代碼
- 'SubRouteName'：搭乘附屬路線名稱 

##### dataframe for dtw

In [16]:
"""
Cell generated by Data Wrangler.
"""
def clean_data(df_TO2A):
    # Drop columns: 'FarePricingType', 'TransferCode' and 14 other columns
    df_TO2A = df_TO2A.drop(columns=['FarePricingType', 'TransferCode', 'IsAbnormal', 'ErrorCode', 'Result', 'TicketCount', 'SrcUpdateTime', 'UpdateTime', 'InfoDate', 'OperatorNo', 'SubRouteUID', 'SubRouteName', 'PaymentPrice', 'DiscountInfo', 'Discount', 'Price'])
    # Drop columns: 'DeboardingStopName', 'BoardingStopName'
    df_TO2A = df_TO2A.drop(columns=['DeboardingStopName', 'BoardingStopName'])
    # Change column type to datetime64[ns] for column: 'BoardingTime'
    df_TO2A = df_TO2A.astype({'BoardingTime': 'datetime64[ns]'})
    # Change column type to datetime64[ns] for column: 'DeboardingTime'
    df_TO2A = df_TO2A.astype({'DeboardingTime': 'datetime64[ns]'})
    return df_TO2A

df_TO2A_clean = clean_data(df_TO2A.copy())

df_TO2A_clean.head()


,Authority,ID,IDType,HolderType,TicketType,SubTicketType,RouteUID,RouteName,Direction,BoardingStopUID,BoardingStopSequence,BoardingTime,DeboardingStopUID,DeboardingStopSequence,DeboardingTime
0,Kaohsiung,68f5c2acceff54ecc52b72b46a1009f7e9c4cde129d9b5...,iPASS,C09,4,#KHH-399,KHH8501,8501(部分繞駛義守大學),1,KHH10850188152,1,2024-07-31 08:00:00,KHH108501131412,3,2024-07-31 09:00:00
1,Kaohsiung,ed357dc217978deca9ee28a7a990ce8dc4ea0a2144d0e8...,EasyCard,A,1,NaN,KHH8505,8505,1,KHH1085059412,10,2024-07-31 11:00:00,KHH108505131452,15,2024-07-31 12:00:00
2,Kaohsiung,447c9a0dc0f11458d19a8cae6aa417ece4d3019ccc935f...,iPASS,C09,4,#KHH-399,KHH8501,8501(部分繞駛義守大學),0,KHH108501115951,2,2024-07-31 14:00:00,KHH108501133051,7,2024-07-31 14:00:00
3,Kaohsiung,c04e31dc2c795becdfc2cea96daf87f71a726db03cbc67...,iPASS,C01,1,NaN,KHH8502,8502,1,KHH10850238332,16,2024-07-31 09:00:00,KHH108502148252,35,2024-07-31 10:00:00
4,Kaohsiung,e98b560912dcc8933eac249d101fc2cb4fae514ed146a6...,EasyCard,A,1,NaN,KHH8501,8501(部分繞駛義守大學),1,KHH10850188152,1,2024-07-31 12:00:00,KHH108501148252,7,2024-07-31 12:00:00


In [18]:
df_TO2A_clean.head(10).to_csv("test_data.csv")